# 02 - Ajuste de Datos ENCSPA 2019
## Preprocesamiento y Limpieza para Machine Learning

**Objetivo:** Preparar los datos de la Encuesta Nacional de Consumo de Sustancias Psicoactivas para entrenamiento de modelos predictivos.

**Variables objetivo identificadas:**
- Marihuana (G_11_F): Variable principal - 8.0% prevalencia
- Cocaína (G_11_G): Variable intermedia - 1.96% prevalencia  
- Basuco (G_11_H): Variable alto riesgo - 0.62% prevalencia

**Variables predictoras clave:**
- Entorno social (G_01, G_02)
- Curiosidad y actitudes (G_03, G_04, G_05)
- Accesibilidad (G_06_A-D, G_07, G_08_A-G)

## Pasos del preprocesamiento (Segundo Commit)

**Fase 1 - Exploración y limpieza básica:**
1. Carga y análisis inicial de los datos
2. Identificación de valores faltantes y outliers
3. Selección de variables relevantes
4. Análisis de distribuciones

**Fase 2 - Transformaciones básicas:**
5. Encoding de variables categóricas
6. Normalización de variables numéricas  
7. División train/test
8. Preparación para modelos (siguiente commit)

*Nota: Balanceo de clases y modelos avanzados se implementarán en commits posteriores*

In [25]:
# Importación de librerías (siguiendo estructura del profesor)
import pandas as pd
import numpy as np # Sin uso actual
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Preprocesamiento y métricas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler # Sin uso actual
from sklearn.impute import SimpleImputer # Sin uso actual
from sklearn.metrics import classification_report, confusion_matrix # Sin uso actual

# Configuración
import warnings
warnings.filterwarnings('ignore')

# Configurar matplotlib para español
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

## 1. Carga y exploración inicial de datos

In [26]:
# Cargar dataset original
df_original = pd.read_csv('../data/g_capitulos.csv')

print(f"Dataset ENCSPA 2019 cargado")
print(f"Dimensiones: {df_original.shape}")
print(f"Observaciones: {df_original.shape[0]:,}")
print(f"Variables: {df_original.shape[1]}")

# Mostrar primeras filas
print(f"\nPrimeras 5 filas del dataset:")
df_original.head()

Dataset ENCSPA 2019 cargado
Dimensiones: (49756, 98)
Observaciones: 49,756
Variables: 98

Primeras 5 filas del dataset:


,G_01,G_01_A,G_02,G_02_A,G_03,G_04,G_05,G_05_A,G_06_A,G_06_B,...,G_14_B,G_14_C,G_14_D,G_14_E,G_14_F,G_14_G,DIRECTORIO,SECUENCIA_ENCUESTA,SECUENCIA_P,ORDEN
0,2,NaN,2,NaN,2,2,2,NaN,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1,2
1,2,NaN,2,NaN,2,2,2,NaN,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,2,1,1,1
2,2,NaN,2,NaN,2,2,2,NaN,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,3,1,1,1
3,2,NaN,2,NaN,2,2,2,NaN,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,4,1,1,1
4,2,NaN,2,NaN,2,2,2,NaN,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,6,1,1,1


## 2. Definición de variables objetivo y predictoras

In [27]:
# Variables seleccionadas basadas en EDA previo
# (Siguiendo metodología del profesor: definir variables clave primero)
# IMPORTANTE: Estas variables deben coincidir exactamente con el notebook EDA

# Variable objetivo principal para este commit
target_variable = 'G_11_F'  # Consumo de marihuana (8% prevalencia)

# Variables predictoras categóricas (entorno social + actitudes)
categorical_features = [
    # Entorno Social
    'G_01',  # Familiares consumen sustancias
    'G_02',  # Amigos consumen sustancias  
    
    # Actitudes y Curiosidad
    'G_03',  # Curiosidad por probar
    'G_04',  # Disposición a consumir
    'G_05'   # Tuvo oportunidad de probar
]

# Variables predictoras numéricas (accesibilidad + exposición)
numerical_features = [
    # Cantidad de familiares/amigos (del EDA)
    'G_01_A',  # Cantidad de familiares que consumen
    'G_02_A',  # Cantidad de amigos que consumen
    
    # Accesibilidad (facilidad de acceso)
    'G_06_A',  # Facilidad acceso marihuana
    'G_06_B',  # Facilidad acceso cocaína
    'G_06_C',  # Facilidad acceso basuco
    'G_06_D',  # Facilidad acceso éxtasis
    
    # Exposición (ofertas recibidas)
    'G_07',    # Ofertas recibidas en último año
    'G_08_A',  # Veces que ofrecieron marihuana
    'G_08_B'   # Veces que ofrecieron cocaína
]

# DICCIONARIO DE NOMBRES LEGIBLES PARA GRÁFICOS Y PRINTS
variable_names = {
    # Variable objetivo
    'G_11_F': 'Consumo de Marihuana',
    
    # Variables categóricas (entorno social + actitudes)
    'G_01': 'Familiares Consumen Sustancias',
    'G_02': 'Amigos Consumen Sustancias',
    'G_03': 'Curiosidad por Probar',
    'G_04': 'Disposición a Consumir',
    'G_05': 'Tuvo Oportunidad de Probar',
    
    # Variables numéricas (cantidades + acceso + exposición)
    'G_01_A': 'Cantidad de Familiares que Consumen',
    'G_02_A': 'Cantidad de Amigos que Consumen',
    'G_06_A': 'Acceso Fácil a Marihuana',
    'G_06_B': 'Acceso Fácil a Cocaína',
    'G_06_C': 'Acceso Fácil a Basuco',
    'G_06_D': 'Acceso Fácil a Éxtasis',
    'G_07': 'Ofertas Recibidas (Último Año)',
    'G_08_A': 'Ofertas de Marihuana',
    'G_08_B': 'Ofertas de Cocaína'
}

# Función para obtener nombre legible
def get_readable_name(var_code):
    """Convierte código de variable a nombre legible"""
    return variable_names.get(var_code, var_code)

# Todas las variables para el análisis (igual que en EDA)
all_features = categorical_features + numerical_features + [target_variable]

print(f"Variable objetivo: {get_readable_name(target_variable)}")
print(f"Variables categóricas: {len(categorical_features)}")
print(f"Variables numéricas: {len(numerical_features)}")
print(f"Total variables seleccionadas: {len(all_features)}")
print(f"Total variables predictoras: {len(categorical_features + numerical_features)}")

print(f"\nVariables categóricas (Entorno + Actitudes):")
for i, var in enumerate(categorical_features, 1):
    print(f"   {i}. {get_readable_name(var)}")

print(f"\nVariables numéricas (Cantidades + Acceso + Exposición):")
for i, var in enumerate(numerical_features, 1):
    print(f"   {i}. {get_readable_name(var)}")

# Verificar disponibilidad de variables
variables_disponibles = [var for var in all_features if var in df_original.columns]
variables_faltantes = [var for var in all_features if var not in df_original.columns]

print(f"\nVariables disponibles: {len(variables_disponibles)}")
if variables_faltantes:
    print(f"Variables faltantes:")
    for var in variables_faltantes:
        print(f"   - {get_readable_name(var)} ({var})")
        
print(f"\nVERIFICACIÓN: Coincidencia con EDA")
print(f"   Aquí tenemos: 1 variable objetivo + {len(categorical_features + numerical_features)} variables predictoras")
print(f"   Variables con nombres legibles configuradas para gráficos")

Variable objetivo: Consumo de Marihuana
Variables categóricas: 5
Variables numéricas: 9
Total variables seleccionadas: 15
Total variables predictoras: 14

Variables categóricas (Entorno + Actitudes):
   1. Familiares Consumen Sustancias
   2. Amigos Consumen Sustancias
   3. Curiosidad por Probar
   4. Disposición a Consumir
   5. Tuvo Oportunidad de Probar

Variables numéricas (Cantidades + Acceso + Exposición):
   1. Cantidad de Familiares que Consumen
   2. Cantidad de Amigos que Consumen
   3. Acceso Fácil a Marihuana
   4. Acceso Fácil a Cocaína
   5. Acceso Fácil a Basuco
   6. Acceso Fácil a Éxtasis
   7. Ofertas Recibidas (Último Año)
   8. Ofertas de Marihuana
   9. Ofertas de Cocaína

Variables disponibles: 15

VERIFICACIÓN: Coincidencia con EDA
   Aquí tenemos: 1 variable objetivo + 14 variables predictoras
   Variables con nombres legibles configuradas para gráficos


## 3. Análisis de calidad de datos

In [28]:
# Crear subset con variables seleccionadas
df = df_original[variables_disponibles].copy()

print("Análisis de calidad de datos")
print("=" * 40)

# 1. Información general
print(f"Dimensiones del subset: {df.shape}")

# 2. Tipos de datos
print(f"\nTipos de datos:")
print(df.dtypes)

# 3. Valores faltantes
print(f"\nAnálisis de valores faltantes:")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_summary = pd.DataFrame({
    'Variable': missing_data.index,
    'Valores_Faltantes': missing_data.values,
    'Porcentaje': missing_percent.values
}).sort_values('Porcentaje', ascending=False)

print(missing_summary)

# 4. Estadísticas descriptivas básicas
print(f"\nDistribución de variable objetivo:")
if target_variable in df.columns:
    target_counts = df[target_variable].value_counts().sort_index()
    print(f"Variable: {target_variable}")
    print(target_counts)
    
    # Calcular prevalencia de consumo
    if 1 in target_counts.index:
        prevalencia = (target_counts[1] / target_counts.sum()) * 100
        print(f"Prevalencia de consumo: {prevalencia:.2f}%")
else:
    print(f"Variable objetivo {target_variable} no encontrada")

Análisis de calidad de datos
Dimensiones del subset: (49756, 15)

Tipos de datos:
G_01        int64
G_02        int64
G_03        int64
G_04        int64
G_05        int64
G_01_A    float64
G_02_A    float64
G_06_A      int64
G_06_B      int64
G_06_C      int64
G_06_D      int64
G_07        int64
G_08_A    float64
G_08_B    float64
G_11_F      int64
dtype: object

Análisis de valores faltantes:
   Variable  Valores_Faltantes  Porcentaje
5    G_01_A              39998   80.388295
6    G_02_A              36191   72.736956
13   G_08_B              35502   71.352199
12   G_08_A              35502   71.352199
1      G_02                  0    0.000000
2      G_03                  0    0.000000
0      G_01                  0    0.000000
3      G_04                  0    0.000000
4      G_05                  0    0.000000
7    G_06_A                  0    0.000000
8    G_06_B                  0    0.000000
10   G_06_D                  0    0.000000
9    G_06_C                  0    0.000000


## 4. Limpieza y filtrado de datos

In [29]:
# Limpieza básica siguiendo metodología del profesor
print("Limpieza y filtrado de datos")
print("=" * 35)

# 1. Eliminar filas con target variable faltante
print(f"Registros originales: {len(df):,}")

# Filtrar solo casos con variable objetivo válida
df_clean = df[df[target_variable].notna()].copy()
print(f"Después de eliminar target faltante: {len(df_clean):,}")

# 2. Convertir variable objetivo a binaria (siguiendo lógica de clasificación)
# 1 = Sí consumió, 2 = No consumió, 9 = No sabe/No responde
print(f"\nDistribución original de variable objetivo:")
print(df_clean[target_variable].value_counts().sort_index())

# Crear variable binaria: 1 = Consumió, 0 = No consumió
# Excluir "No sabe/No responde" (código 9)
df_clean = df_clean[df_clean[target_variable] != 9].copy()
df_clean['target_binary'] = (df_clean[target_variable] == 1).astype(int)

print(f"\nDespués de excluir 'No sabe': {len(df_clean):,}")
print(f"Distribución binaria:")
print(df_clean['target_binary'].value_counts())

# 3. Verificar prevalencia final
prevalencia_final = df_clean['target_binary'].mean() * 100
print(f"\nPrevalencia final de consumo: {prevalencia_final:.2f}%")

# 4. Información del dataset limpio
print(f"\nResumen del dataset limpio:")
print(f"   Dimensiones: {df_clean.shape}")
print(f"   Casos válidos: {len(df_clean):,}")
print(f"   Casos positivos: {df_clean['target_binary'].sum():,}")
print(f"   Casos negativos: {(1-df_clean['target_binary']).sum():,}")

Limpieza y filtrado de datos
Registros originales: 49,756
Después de eliminar target faltante: 49,756

Distribución original de variable objetivo:
G_11_F
1     3982
2    45774
Name: count, dtype: int64

Después de excluir 'No sabe': 49,756
Distribución binaria:
target_binary
0    45774
1     3982
Name: count, dtype: int64

Prevalencia final de consumo: 8.00%

Resumen del dataset limpio:
   Dimensiones: (49756, 16)
   Casos válidos: 49,756
   Casos positivos: 3,982
   Casos negativos: 45,774


## 5. Análisis exploratorio de variables predictoras

In [30]:
# Análisis de variables predictoras (metodología exploratoria del profesor)
print("Análisis de variables predictoras")
print("=" * 40)

# 1. Variables categóricas - Análisis de distribuciones
print("Variables categóricas (Entorno Social + Actitudes):")
for var in categorical_features:
    if var in df_clean.columns:
        print(f"\n--- {get_readable_name(var)} ---")
        distribucion = df_clean[var].value_counts().sort_index()
        print(distribucion)
        
        # Mostrar distribución porcentual
        distribucion_pct = (distribucion / distribucion.sum()) * 100
        print("Porcentajes:")
        for idx, pct in distribucion_pct.items():
            print(f"  {idx}: {pct:.1f}%")

# 2. Variables numéricas - Estadísticas descriptivas
print(f"\nVariables numéricas (Cantidades + Acceso + Exposición):")
numeric_vars_available = [var for var in numerical_features if var in df_clean.columns]

if numeric_vars_available:
    # Mostrar estadísticas con nombres legibles
    print("\nEstadísticas descriptivas:")
    for var in numeric_vars_available:
        stats = df_clean[var].describe()
        print(f"\n{get_readable_name(var)}:")
        print(f"  Media: {stats['mean']:.2f}")
        print(f"  Mediana: {stats['50%']:.2f}")
        print(f"  Desv. Estándar: {stats['std']:.2f}")
        print(f"  Min-Max: {stats['min']:.0f} - {stats['max']:.0f}")
    
    # Verificar valores únicos para detectar si son realmente categóricas
    print(f"\nValores únicos por variable:")
    for var in numeric_vars_available:
        unique_vals = sorted(df_clean[var].dropna().unique())
        unique_count = len(unique_vals)
        if unique_count <= 10:  # Mostrar valores si son pocos
            print(f"{get_readable_name(var)}: {unique_vals}")
        else:  # Solo mostrar cantidad si son muchos
            print(f"{get_readable_name(var)}: {unique_count} valores únicos (rango: {min(unique_vals)}-{max(unique_vals)})")

# 3. Correlación preliminar con variable objetivo
print(f"\nRelación con variable objetivo:")
print("Variables más correlacionadas con consumo de marihuana:")

# Crear DataFrame para correlaciones
df_corr = df_clean.copy()

# Convertir categóricas para correlación rápida
for var in categorical_features:
    if var in df_corr.columns:
        # Convertir a binario para análisis rápido (1=Sí, 0=Resto)
        df_corr[f"{var}_binary"] = (df_corr[var] == 1).astype(int)

# Calcular correlaciones
correlation_vars = [f"{var}_binary" for var in categorical_features if var in df_clean.columns]
correlation_vars.extend([var for var in numerical_features if var in df_clean.columns])

if correlation_vars:
    correlations = df_corr[correlation_vars + ['target_binary']].corr()['target_binary'].drop('target_binary')
    correlations_sorted = correlations.abs().sort_values(ascending=False)
    
    print("Top 10 correlaciones (valor absoluto):")
    for i, (var, corr) in enumerate(correlations_sorted.head(10).items(), 1):
        # Limpiar nombre de variable (quitar _binary si existe)
        clean_var = var.replace('_binary', '')
        readable_name = get_readable_name(clean_var)
        print(f"  {i}. {readable_name}: {corr:.3f}")
        
# Guardar variables de correlación para visualizaciones
globals()['correlation_vars'] = correlation_vars
globals()['correlations_sorted'] = correlations_sorted

Análisis de variables predictoras
Variables categóricas (Entorno Social + Actitudes):

--- Familiares Consumen Sustancias ---
G_01
1     9758
2    39998
Name: count, dtype: int64
Porcentajes:
  1: 19.6%
  2: 80.4%

--- Amigos Consumen Sustancias ---
G_02
1    13565
2    36191
Name: count, dtype: int64
Porcentajes:
  1: 27.3%
  2: 72.7%

--- Curiosidad por Probar ---
G_03
1     8188
2    41568
Name: count, dtype: int64
Porcentajes:
  1: 16.5%
  2: 83.5%

--- Disposición a Consumir ---
G_04
1     2595
2    47161
Name: count, dtype: int64
Porcentajes:
  1: 5.2%
  2: 94.8%

--- Tuvo Oportunidad de Probar ---
G_05
1    13545
2    36211
Name: count, dtype: int64
Porcentajes:
  1: 27.2%
  2: 72.8%

Variables numéricas (Cantidades + Acceso + Exposición):

Estadísticas descriptivas:

Cantidad de Familiares que Consumen:
  Media: 1.46
  Mediana: 1.00
  Desv. Estándar: 0.50
  Min-Max: 1 - 2

Cantidad de Amigos que Consumen:
  Media: 1.81
  Mediana: 2.00
  Desv. Estándar: 0.39
  Min-Max: 1 - 2

Ac

## 6. Preparación básica para machine learning

In [31]:
# Preparación básica para ML (siguiendo estructura del profesor)
print("Preparación básica para Machine Learning")
print("=" * 45)

# 1. Seleccionar features válidas para el modelo
features_para_modelo = []

# Agregar variables categóricas disponibles
for var in categorical_features:
    if var in df_clean.columns:
        features_para_modelo.append(var)

# Agregar variables numéricas disponibles  
for var in numerical_features:
    if var in df_clean.columns:
        features_para_modelo.append(var)

print(f"Features seleccionadas para modelo: {len(features_para_modelo)}")
print(f"\nVariables categóricas incluidas:")
cat_included = [var for var in categorical_features if var in df_clean.columns]
for i, var in enumerate(cat_included, 1):
    print(f"   {i}. {get_readable_name(var)}")

print(f"\nVariables numéricas incluidas:")
num_included = [var for var in numerical_features if var in df_clean.columns]
for i, var in enumerate(num_included, 1):
    print(f"   {i}. {get_readable_name(var)}")

# 2. Crear matrices X e y
X = df_clean[features_para_modelo].copy()
y = df_clean['target_binary'].copy()

print(f"\nDimensiones finales:")
print(f"   X (features): {X.shape}")
print(f"   y (target): {y.shape}")

# 3. Verificar valores faltantes en features
print(f"\nValores faltantes en features:")
missing_in_features = X.isnull().sum()
if missing_in_features.sum() > 0:
    print("Variables con datos faltantes:")
    for var, missing in missing_in_features[missing_in_features > 0].items():
        pct_missing = (missing / len(X)) * 100
        print(f"  - {get_readable_name(var)}: {missing} casos ({pct_missing:.1f}%)")
else:
    print("No hay valores faltantes en las features seleccionadas")

# 4. División train/test básica (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Mantener proporción de clases
)

print(f"\nDivisión de datos completada:")
print(f"   Entrenamiento: {X_train.shape[0]:,} muestras")
print(f"   Prueba: {X_test.shape[0]:,} muestras")
print(f"   Prevalencia train: {y_train.mean()*100:.1f}%")
print(f"   Prevalencia test: {y_test.mean()*100:.1f}%")

# 5. Resumen de variables más importantes para el modelo
if 'correlations_sorted' in globals():
    print(f"\nTop 5 variables más predictivas:")
    for i, (var, corr) in enumerate(correlations_sorted.head(5).items(), 1):
        clean_var = var.replace('_binary', '')
        readable_name = get_readable_name(clean_var)
        print(f"   {i}. {readable_name} (correlación: {corr:.3f})")

# 6. Guardar información para siguientes pasos
print(f"\nDatos preparados para siguiente commit:")
print(f"   Variables objetivo y predictoras definidas con nombres legibles")
print(f"   Datos limpios y filtrados") 
print(f"   División train/test realizada")
print(f"   {len(features_para_modelo)} features listas para modelos")
print(f"   Pendiente: Encoding, normalización y modelos")

Preparación básica para Machine Learning
Features seleccionadas para modelo: 14

Variables categóricas incluidas:
   1. Familiares Consumen Sustancias
   2. Amigos Consumen Sustancias
   3. Curiosidad por Probar
   4. Disposición a Consumir
   5. Tuvo Oportunidad de Probar

Variables numéricas incluidas:
   1. Cantidad de Familiares que Consumen
   2. Cantidad de Amigos que Consumen
   3. Acceso Fácil a Marihuana
   4. Acceso Fácil a Cocaína
   5. Acceso Fácil a Basuco
   6. Acceso Fácil a Éxtasis
   7. Ofertas Recibidas (Último Año)
   8. Ofertas de Marihuana
   9. Ofertas de Cocaína

Dimensiones finales:
   X (features): (49756, 14)
   y (target): (49756,)

Valores faltantes en features:
Variables con datos faltantes:
  - Cantidad de Familiares que Consumen: 39998 casos (80.4%)
  - Cantidad de Amigos que Consumen: 36191 casos (72.7%)
  - Ofertas de Marihuana: 35502 casos (71.4%)
  - Ofertas de Cocaína: 35502 casos (71.4%)

División de datos completada:
   Entrenamiento: 39,804 muestr

## 7. Visualización exploratoria básica

In [32]:
# Visualización exploratoria básica (usando plotly como el profesor)
print("Visualización exploratoria básica")
print("=" * 40)

# 1. Distribución de la variable objetivo
fig_target = px.bar(
    x=['No Consume', 'Consume'], 
    y=[len(y) - y.sum(), y.sum()],
    title=f'Distribución de {get_readable_name(target_variable)}',
    labels={'x': 'Categoría', 'y': 'Número de casos'},
    color=[len(y) - y.sum(), y.sum()],
    color_continuous_scale='Blues',
    text=[f'{len(y) - y.sum():,}', f'{y.sum():,}']
)

fig_target.update_traces(textposition='outside')
fig_target.update_layout(height=400, showlegend=False)
fig_target.show()

# 2. Análisis de variables categóricas más importantes
if 'correlations_sorted' in globals() and len(correlation_vars) >= 2:
    # Seleccionar las 2 variables categóricas más correlacionadas
    top_categorical = correlations_sorted.head(3).index.tolist()
    
    # Filtrar solo variables categóricas para el gráfico
    categorical_vars_binary = [var for var in top_categorical if '_binary' in var][:2]
    
    if len(categorical_vars_binary) >= 2:
        # Crear gráfico para variables categóricas top
        fig_cat = make_subplots(
            rows=1, cols=2,
            subplot_titles=[get_readable_name(var.replace('_binary', '')) for var in categorical_vars_binary]
        )
        
        for i, var in enumerate(categorical_vars_binary):
            original_var = var.replace('_binary', '')
            if original_var in df_clean.columns:
                counts = df_clean[original_var].value_counts().sort_index()
                
                # Crear etiquetas más legibles para los valores
                labels = []
                for idx in counts.index:
                    if idx == 1:
                        labels.append('Sí')
                    elif idx == 2:
                        labels.append('No')
                    elif idx == 9:
                        labels.append('No sabe')
                    else:
                        labels.append(str(idx))
                
                fig_cat.add_trace(
                    go.Bar(
                        x=labels, 
                        y=counts.values, 
                        name=get_readable_name(original_var),
                        text=[f'{v:,}' for v in counts.values],
                        textposition='outside'
                    ),
                    row=1, col=i+1
                )
        
        fig_cat.update_layout(
            title_text="Variables Más Predictivas del Entorno Social",
            height=400,
            showlegend=False
        )
        fig_cat.show()

# 3. Matriz de correlación básica (solo top variables)
if 'correlations_sorted' in globals() and len(correlation_vars) >= 3:
    # Seleccionar top 6 variables para matriz de correlación
    top_vars = correlations_sorted.head(6).index.tolist()
    top_vars.append('target_binary')
    
    # Crear matriz de correlación
    corr_matrix = df_corr[top_vars].corr()
    
    # Crear etiquetas legibles para la matriz
    readable_labels = []
    for var in corr_matrix.columns:
        if var == 'target_binary':
            readable_labels.append('Consumo Marihuana')
        else:
            clean_var = var.replace('_binary', '')
            readable_labels.append(get_readable_name(clean_var))
    
    # Crear gráfico con nombres legibles
    fig_corr = px.imshow(
        corr_matrix,
        x=readable_labels,
        y=readable_labels,
        title='Matriz de Correlación - Variables Más Predictivas',
        color_continuous_scale='RdBu_r',
        aspect='auto',
        text_auto='.2f'
    )
    
    fig_corr.update_layout(
        height=600, 
        width=700,
        xaxis_title="Variables Predictoras",
        yaxis_title="Variables Predictoras"
    )
    fig_corr.show()

# 4. Gráfico de barras con correlaciones top
if 'correlations_sorted' in globals():
    top_corr = correlations_sorted.head(8)
    
    # Crear nombres legibles para correlaciones
    readable_names = []
    correlations_values = []
    
    for var, corr in top_corr.items():
        clean_var = var.replace('_binary', '')
        readable_names.append(get_readable_name(clean_var))
        correlations_values.append(corr)
    
    fig_corr_bar = px.bar(
        x=correlations_values,
        y=readable_names,
        title='Variables Más Correlacionadas con Consumo de Marihuana',
        labels={'x': 'Correlación (Valor Absoluto)', 'y': 'Variable Predictora'},
        color=correlations_values,
        color_continuous_scale='Reds',
        orientation='h',
        text=[f'{v:.3f}' for v in correlations_values]
    )
    
    fig_corr_bar.update_traces(textposition='outside')
    fig_corr_bar.update_layout(
        height=500,
        showlegend=False,
        yaxis={'categoryorder': 'total ascending'}
    )
    fig_corr_bar.show()

print("Visualizaciones básicas completadas")
print("Todas las gráficas usan nombres legibles y entendibles")
print("Análisis más detallado se realizará en próximos commits")

Visualización exploratoria básica


Visualizaciones básicas completadas
Todas las gráficas usan nombres legibles y entendibles
Análisis más detallado se realizará en próximos commits
